# VistaMilk data challenge

## Data description
Data used in this study originated from Teagasc Moorepark Dairy Research Farm (Fermoy, Co. Cork, Ireland) between May and August in 2015, 2016, and 2017. A total of 120 HolsteinFriesian cows from different parities were involved in the experiment across the years, with a mean number of 36 samples per cow, and with some of the cows participating in the experiment in more than 1 yr. Each year, 54 cows were randomly assigned to different dietary treatments for the entire lactation period. The treatment diets included grass (GRS), which consisted of cows maintained outdoors on a perennial ryegrass sward only, clover (CLV), where cows were maintained outdoors on a perennial ryegrass white clover sward (with an annual average clover content of 20%) only, and TMR, where cows were maintained indoors and fed with a single nutritional mix containing grass silage, maize silage, and concentrates. Further information on the experimental design and dietary treatments have been described by O’Callaghan et al. (2016). The cows were milked twice daily (0730 and 1530 h), and a.m. and p.m. milk samples were collected once weekly from consecutive milkings and analysed by a Pro-Foss FT6000 (FOSS). A total of 4,364 milk spectra were stored, comprising 1,060 wavelengths in the region from 925 cm−1 and 5,010 cm−1. The wavelengths values were recorded as transmittance values.

## tl;dr
This is a classification problem where 3 classes are targeted:

- GRS
- CLV
- TMR

Data are sampled from 54 cows across 3 years. Each data points contain 1060 wavelengths within a specified light region.

In [ ]:
import sys
import importlib

from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import rich

sys.path.insert(0, '..')
from src import utils

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

from PIL import Image

importlib.reload(utils)

In [ ]:
# read data in
# make the conversion only one as the xlsx files take forever to load
full_data = Path('..', 'data', 'full_dataset.xlsx')
train_path = Path('..', 'data', 'raw_train.csv')
test_path = Path('..', 'data', 'raw_test.csv')

train_df = utils.read_raw_data(full_data, train_path, 0)
test_df = utils.read_raw_data(full_data, test_path, 1)

In [ ]:
no_samples = train_df.shape[0]
no_features = train_df.shape[1] - 1

rich.print(f'Total samples: {no_samples}')
rich.print(f'Total features: {no_features}')

In [ ]:
train_df.head(10)

In [ ]:
test_df.head(10)

In [ ]:
train_df.describe().T

In [ ]:
test_df.describe().T

In [ ]:
f = utils.plot_wave(train_df.iloc[10], target_col='Diet')
f.show()

In [ ]:
clv = train_df[train_df['Diet'] == 'CLV'].drop(['Diet'], axis=1)
clv_centroid = np.mean(clv.values, axis=0)

grs = train_df[train_df['Diet'] == 'GRS'].drop(['Diet'], axis=1)
grs_centroid = np.mean(grs.values, axis=0)

tmr = train_df[train_df['Diet'] == 'TMR'].drop(['Diet'], axis=1)
tmr_centroid = np.mean(grs.values, axis=0)

fig = plt.figure(figsize=(25, 10))

plt.plot(clv_centroid, label='CLV centroid')
plt.plot(grs_centroid, label='GRS centroid')
plt.plot(tmr_centroid, label='TMR centroid')

plt.grid()
plt.legend()
plt.show()

In [ ]:
clv = train_df[train_df['Diet'] == 'CLV'].drop(['Diet'], axis=1)
clv_centroid = np.median(clv.values, axis=0)

grs = train_df[train_df['Diet'] == 'GRS'].drop(['Diet'], axis=1)
grs_centroid = np.median(grs.values, axis=0)

tmr = train_df[train_df['Diet'] == 'TMR'].drop(['Diet'], axis=1)
tmr_centroid = np.median(grs.values, axis=0)

fig = plt.figure(figsize=(25, 10))

plt.plot(clv_centroid, label='CLV centroid')
plt.plot(grs_centroid, label='GRS centroid')
plt.plot(tmr_centroid, label='TMR centroid')

plt.grid()
plt.legend()
plt.show()

In [ ]:
clv = train_df[train_df['Diet'] == 'CLV'].drop(['Diet'], axis=1)
clv_centroid = np.mean(clv.values, axis=0)
clv_centroid_s = np.std(clv.values, axis=0)

grs = train_df[train_df['Diet'] == 'GRS'].drop(['Diet'], axis=1)
grs_centroid = np.std(grs.values, axis=0)

tmr = train_df[train_df['Diet'] == 'TMR'].drop(['Diet'], axis=1)
tmr_centroid = np.std(grs.values, axis=0)

fig = plt.figure(figsize=(25, 10))

plt.plot(clv_centroid, label='CLV centroid')
plt.plot(clv_centroid_s, label='CLV centroid std')
# plt.plot(grs_centroid, label='GRS centroid')
# plt.plot(tmr_centroid, label='TMR centroid')

plt.vlines(172, ymin=0, ymax=1, color='red', linestyle='--')
plt.vlines(205, ymin=0, ymax=1, color='red', linestyle='--')

plt.vlines(536, ymin=0, ymax=1, color='red', linestyle='--')
plt.vlines(728, ymin=0, ymax=1, color='red', linestyle='--')

plt.vlines(748, ymin=0, ymax=1, color='red', linestyle='--')
plt.vlines(1059, ymin=0, ymax=1, color='red', linestyle='--')

plt.grid()
plt.legend()
plt.show()

In [ ]:
rich.print(clv_centroid.std())

## Missing values
Check if there are any missing values within the train and the test sets.

In [ ]:
total_train_missing = sum(train_df.isnull().sum())
total_test_missing = sum(test_df.isnull().sum())

rich.print(f'Train missing values: {total_train_missing}')
rich.print(f'Test missing values: {total_test_missing}')

## Target variable: `Diet` column

In [ ]:
target = train_df['Diet']
unique_target_values = target.unique()
target_stats = {}

for target_value in unique_target_values:
    target_stats[target_value] = {
        'count': train_df[train_df['Diet'] == target_value].shape[0],
        'perc': train_df[train_df['Diet'] == target_value].shape[0] / train_df.shape[0]
    }

rich.print(f'Unique values in the target column: {unique_target_values}')
print()

for t, v in target_stats.items():
    count = v['count']
    perc = v['perc']
    rich.print(f'Target {t}: {count} instances ({perc}% of total)')

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.hist(target)
plt.grid()
plt.show()

In [ ]:
missing_targets = target.isnull().sum()
rich.print(f'Missing labels: {missing_targets}')

## Basic model training
These training examples are based on a stratified 5-fold cross validation.

In [ ]:
# data shuffling here: move somewhere else
train_df = train_df.sample(frac=1)

X_train = train_df.drop(['Diet'], axis=1)
y = train_df['Diet']

In [ ]:
classifiers = {
    # 'knn': KNeighborsClassifier(),
    # 'ridge': RidgeClassifierCV(alphas=np.logspace(-3, 3, 10)),
    'tree': DecisionTreeClassifier(),
    # 'forest': RandomForestClassifier(),
    # 'SVM': SVC(),
    # 'adaboost': AdaBoostClassifier(),
    # 'mlp': MLPClassifier()
}

for cls_name, classifier in classifiers.items():
    score, pred, report, confusion = utils.basics(classifier, X_train, y)
    
    rich.print(f'[bold underline magenta]{cls_name}[/bold underline magenta]')
    rich.print(f'Scores: {score} (avg. {score.mean()})')
    rich.print(report)
    rich.print(confusion)
    print()

## Convolutional approach
Treat the waves as images, then use regular CNNs + FCN to classify them.

In [ ]:
# to make the wave image-like, the dimensions should be 33x33
# (1060 original size, closest square is 1089 = 33^2)
# (for RGB conversion, the resulting image would only be 19x19 with a padding of 23)
padded_df = X_train.copy()

for i in range(29):
    wave = 1061 + i
    padded_df[f'col{wave}'] = 0

img_like = padded_df.iloc[10].values

# img_like *= 255 / img_like.max()
img_like = img_like.reshape((33, 33))

plt.imshow(img_like, cmap='Greys')
plt.colorbar()

im = Image.fromarray(img_like)
im = im.convert('RGB')
im.save('test.png')